In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/sg-kopi/english_vocab.Dictionary.pkl
/kaggle/input/sg-kopi/singlish_vocab.Dictionary.pkl
/kaggle/input/sg-kopi/kopitiam.csv
/kaggle/input/sg-kopi/decoder_attention_100_100000_0.5.pkl
/kaggle/input/sg-kopi/decoder_vanilla_100_100000.pkl
/kaggle/input/sg-kopi/decoder_vanilla_100_100000_0.5.pkl
/kaggle/input/sg-kopi/encoder_vanilla_100_100000_0.5.pkl
/kaggle/input/sg-kopi/encoder_attention_100_100000_0.5.pkl
/kaggle/input/sg-kopi/encoder_vanilla_100_100000.pkl


## Intro
I am a coffee lover. Any kind of coffee, from American, Japanese, Italian, Southeast Asian, and others. Graduated from MIT, I will open a coffee shop in Singapore beside my work as ML engineer. The menus was listed in the data below.

The society likes coffee and tea very much. I was wondering why they are much smile. Everyone like my coffee, but we have limited stock.

People order our menu with Singlish or English, so we need to adjust.

In [2]:
menu = pd.read_csv('/kaggle/input/sg-kopi/kopitiam.csv')
menu

,Singlish,English,Source
0,Kopi O,Black Coffee with Sugar,https://daneshd.com/2010/02/28/a-rough-guide-t...
1,Kopi,Black Coffee with Condensed Milk,https://daneshd.com/2010/02/28/a-rough-guide-t...
2,Kopi C,Black Coffee with Evaporated Milk,https://daneshd.com/2010/02/28/a-rough-guide-t...
3,Kopi Kosong,Black Coffee without sugar or milk,https://daneshd.com/2010/02/28/a-rough-guide-t...
4,Kopi Gah Dai,Black Coffee with extra condensed milk,https://daneshd.com/2010/02/28/a-rough-guide-t...
...,...,...,...
163,Teh tarik,the Malay tea described above,https://peregrinatewithme.wordpress.com/2013/1...
164,Teh halia,tea with ginger water,https://peregrinatewithme.wordpress.com/2013/1...
165,Teh peng,tea with ice,https://peregrinatewithme.wordpress.com/2013/1...
166,Teh siu dai,tea with milk and less sugar,https://peregrinatewithme.wordpress.com/2013/1...


## 1. Extracting ingredients
###### Forget NLP, is hard, just use regex split 

In [3]:
import re
subs = []
leng = []
for i in range(menu['English'].shape[0]):
    a = menu['English'][i]
    sep1 = 'without'
    ingredient = a.split(sep1, 1)[0]
    ingredient = re.split('with | but | and|,',ingredient)
    print(i)
    print(menu['Singlish'][i], ingredient, len(ingredient))
    subs.append(ingredient)
    leng.append(len(ingredient))
    


0
Kopi O ['Black Coffee ', 'Sugar'] 2
1
Kopi ['Black Coffee ', 'Condensed Milk'] 2
2
Kopi C ['Black Coffee ', 'Evaporated Milk'] 2
3
Kopi Kosong ['Black Coffee '] 1
4
Kopi Gah Dai ['Black Coffee ', 'extra condensed milk'] 2
5
Kopi Siew Dai ['Coffee ', 'Condensed Milk', 'less sugar'] 3
6
Kopi O Siew Dai ['Black Coffee ', 'less sugar'] 2
7
Kopi Po ['Coffee ', 'Condensed Milk', 'weaker (they add more water)'] 3
8
Kopi O Po ['Black Coffee ', 'Sugar', 'weaker (they add more water)'] 3
9
Kopi Gau ['Strong Coffee ', 'Condensed Milk'] 2
10
Kopi O Gau ['Strong Black Coffee ', 'Sugar'] 2
11
Kopi Peng ['Iced Coffee ', 'Condensed Milk'] 2
12
Kopi O Peng ['Iced Black Coffee ', 'sugar'] 2
13
Kopi Gau Peng ['Strong Iced Coffee ', 'Condensed Milk'] 2
14
Kopi O Gau Peng ['Strong Black Coffee ', 'Sugar'] 2
15
Kopi Kosong Peng ['Iced black coffee '] 1
16
Kopi Gau Kosong Peng ['Iced Strong Black coffee '] 1
17
Kopi ['Hot coffee ', 'condensed milk', ' sugar'] 3
18
Kopi Siew Dai ['Hot coffee ', 'condensed m

In [4]:
contain = pd.DataFrame(subs, columns = ['Ingredient 1','Ingredient 2','Ingredient 3','Ingredient 4','Ingredient 5'])
length  = pd.DataFrame(leng, columns = ['Ingredient count'])
contain['Ingredient 1'] = contain['Ingredient 1'].str.strip()
contain['Ingredient 2'] = contain['Ingredient 2'].str.strip()
contain['Ingredient 3'] = contain['Ingredient 3'].str.strip()
contain['Ingredient 4'] = contain['Ingredient 4'].str.strip()
contain['Ingredient 5'] = contain['Ingredient 5'].str.strip()

In [5]:
menu = menu.join(contain)
menu = menu.join(length)

In [6]:
menu = menu.drop('Source',axis = 1)

In [7]:
print(menu['Ingredient 1'].value_counts(), '\n\n')
print(menu['Ingredient 2'].value_counts(), '\n\n')
print(menu['Ingredient 3'].value_counts(), '\n\n')
print(menu['Ingredient 4'].value_counts(), '\n\n')
print(menu['Ingredient 5'].value_counts(), '\n\n')

Hot coffee                    20
Tea                           17
Hot tea                       13
Hot black coffee              10
coffee                        10
                              ..
Stronger tea concentration     1
Iced water                     1
Very strong tea                1
Iced Strong Black coffee       1
Iced version of tea            1
Name: Ingredient 1, Length: 65, dtype: int64 


condensed milk                                       30
evaporated milk                                      24
less sugar                                            6
Condensed Milk                                        6
milk                                                  5
sugar only                                            4
sugar                                                 4
more sugar                                            4
Sugar                                                 4
more coffee powder                                    3
lesser coffee powder         

In [8]:
menu.fillna(value=pd.np.nan, inplace=True)


In [9]:
menu.head(5)

,Singlish,English,Ingredient 1,Ingredient 2,Ingredient 3,Ingredient 4,Ingredient 5,Ingredient count
0,Kopi O,Black Coffee with Sugar,Black Coffee,Sugar,NaN,NaN,NaN,2
1,Kopi,Black Coffee with Condensed Milk,Black Coffee,Condensed Milk,NaN,NaN,NaN,2
2,Kopi C,Black Coffee with Evaporated Milk,Black Coffee,Evaporated Milk,NaN,NaN,NaN,2
3,Kopi Kosong,Black Coffee without sugar or milk,Black Coffee,NaN,NaN,NaN,NaN,1
4,Kopi Gah Dai,Black Coffee with extra condensed milk,Black Coffee,extra condensed milk,NaN,NaN,NaN,2


## 2. Set Price

We will set price based on how much ingredients used in beverages. Because I am generous, I will set each ingredient for 1 SGD. 

In [10]:
menu['price (SGD)'] = menu['Ingredient count']

In [11]:
menu.head(5)

,Singlish,English,Ingredient 1,Ingredient 2,Ingredient 3,Ingredient 4,Ingredient 5,Ingredient count,price (SGD)
0,Kopi O,Black Coffee with Sugar,Black Coffee,Sugar,NaN,NaN,NaN,2,2
1,Kopi,Black Coffee with Condensed Milk,Black Coffee,Condensed Milk,NaN,NaN,NaN,2,2
2,Kopi C,Black Coffee with Evaporated Milk,Black Coffee,Evaporated Milk,NaN,NaN,NaN,2,2
3,Kopi Kosong,Black Coffee without sugar or milk,Black Coffee,NaN,NaN,NaN,NaN,1,1
4,Kopi Gah Dai,Black Coffee with extra condensed milk,Black Coffee,extra condensed milk,NaN,NaN,NaN,2,2


## 3. Buy Ingredient Stock

To operate, we need to buy the ingredient, and it will be used for selling. I assume there will be direct restock if the inventory was empty for each ingredient, because we won't disappoint the customer. 

    For 1st ingredient, 100 stock
    For 2nd ingredient, 50 stock
    For 3rd ingredient, 30 stock
    For 4th ingredient, 20 stock
    For 5th ingredient, 10 stock


In [12]:
menu['Ingredient 1 stock'] =100
menu['Ingredient 2 stock'] =50
menu['Ingredient 3 stock'] =30
menu['Ingredient 4 stock'] =20
menu['Ingredient 5 stock'] =10

In [13]:
menu.head(5)

,Singlish,English,Ingredient 1,Ingredient 2,Ingredient 3,Ingredient 4,Ingredient 5,Ingredient count,price (SGD),Ingredient 1 stock,Ingredient 2 stock,Ingredient 3 stock,Ingredient 4 stock,Ingredient 5 stock
0,Kopi O,Black Coffee with Sugar,Black Coffee,Sugar,NaN,NaN,NaN,2,2,100,50,30,20,10
1,Kopi,Black Coffee with Condensed Milk,Black Coffee,Condensed Milk,NaN,NaN,NaN,2,2,100,50,30,20,10
2,Kopi C,Black Coffee with Evaporated Milk,Black Coffee,Evaporated Milk,NaN,NaN,NaN,2,2,100,50,30,20,10
3,Kopi Kosong,Black Coffee without sugar or milk,Black Coffee,NaN,NaN,NaN,NaN,1,1,100,50,30,20,10
4,Kopi Gah Dai,Black Coffee with extra condensed milk,Black Coffee,extra condensed milk,NaN,NaN,NaN,2,2,100,50,30,20,10


## 4. Operational hour!

Customer were coming, they order random items, using Singlish or English name, at random quantity. I assume they pay with e-money so no changes. 

There will be 100 Customer!

In [14]:
order_singlish = menu['Singlish'].tolist()
order_english = menu['English'].tolist()
order = np.concatenate((order_singlish, order_english))

In [15]:
print('Menus: ',order)

Menus:  ['Kopi O' 'Kopi' 'Kopi C' 'Kopi Kosong' 'Kopi Gah Dai' 'Kopi Siew Dai'
 'Kopi O Siew Dai' 'Kopi Po' 'Kopi O Po' 'Kopi Gau' 'Kopi O Gau'
 'Kopi Peng' 'Kopi O Peng' 'Kopi Gau Peng' 'Kopi O Gau Peng'
 'Kopi Kosong Peng' 'Kopi Gau Kosong Peng' 'Kopi' 'Kopi Siew Dai'
 'Kopi Ga Dai' 'Kopi Po' 'Kopi Gau' 'Kopi Po Siew Dai' 'Kopi Po Ga Dai'
 'Kopi Gau Siew Dai' 'Kopi Gau Ga Dai' 'Kopi Kosong' 'Kopi C'
 'Kopi C Siew Dai' 'Kopi C Ga Dai' 'Kopi C Po' 'Kopi C Gau'
 'Kopi C Po Siew Dai' 'Kopi C Po Ga Dai' 'Kopi C Gau Siew Dai'
 'Kopi C Gau Ga Dai' 'Kopi C Kosong' 'Kopi O' 'Kopi O Siew Dai'
 'Kopi O Ga Dai' 'Kopi O Po' 'Kopi O Gau' 'Kopi O Po Siew Dai'
 'Kopi O Po Ga Dai' 'Kopi O Gau Siew Dai' 'Kopi O Gau Ga Dai'
 'Kopi O Kosong' 'Teh' 'Teh Siew Dai' 'Teh Ga Dai' 'Teh Kosong'
 'Teh Tarik' 'Teh Halia' 'Teh C' 'Teh C Siew Dai' 'Teh C Ga Dai'
 'Teh C Kosong' 'Teh C Tarik' 'Teh O' 'Teh O Siew Dai' 'Teh O Ga Dai'
 'Teh O Kosong' 'Teh O Tarik' 'Tiao Her' 'Tut Kiu' 'Milo Kosong'
 'Milo Dinosaur' 'T

The customer specify how many kind of beverages, then the number for each beverages.

In [16]:
buy = []
many = []
import random
random.seed(42)
for i in range(100):
    kind = random.randint(1,3)
    for k in range(kind):
        pick = random.choice(order)
        buy.append(pick)
        q = random.randint(1,4)
        many.append(q)
        
    print(pick,q)


Teh O ice limau 2
Kopi Gau Kosong Peng 1
Teh O kah dai 2
tea with ginger water 4
Hot tea without milk and sugar 3
Beer Gao 4
Teh O 3
Teh 4
Iced black coffee without sugar 3
Hot coffee with condensed milk and more coffee powder and more sugar 1
Kopi C Gau Ga Dai 1
Teh Tarik 4
Teh C 3
the Malay tea described above 1
Teh C 4
Teh siu dai 1
Kopi Gau Kosong Peng 3
Teh C kah dai 3
tea with milk and more sugar 4
Da Jia Hao 3
Milo Peng 3
Hot coffee with condensed milk and sugar 2
Coffee with evaporated milk and sugar instead of condensed milk 4
Kopi Gau Siew Dai 1
coffee without sugar or milk 2
Tea with less sugar 4
tea with milk and no sugar 3
Kopi C 4
Hot Milo with condensed milk and sugar 1
coffee with sugar only 3
Teh O 1
Hot coffee with condensed milk and sugar 3
Kopi C Ga Dai 2
Weak or thin tea with condensed milk 2
100 plus 2
Teh C kah dai 2
Hot tea with evaporated milk only 4
Milo Peng 2
Coffee with Condensed Milk but less sugar 1
Kopi Kosong 1
Kopi Gau Kosong Peng 3
Simply cofee withou

In [17]:
buy = pd.DataFrame(buy, columns = ['Ordered beverage'])
many = pd.DataFrame(many, columns = ['Counts'])

In [18]:
buy = buy.join(many)

In [19]:
buy.head(10)

,Ordered beverage,Counts
0,Teh C Tarik,1
1,Kopi siu dai,2
2,Teh O ice limau,2
3,Teh Halia,1
4,Extra sweet coffee with condensed milk,4
5,Kopi Gau Kosong Peng,1
6,Teh O kah dai,2
7,coffee with less sugar,1
8,Hot Horlicks,2
9,tea with ginger water,4


## 5. Calculate Revenue

Time to close. We need to calculate how much we got. (To be continued)